In [1]:
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/models"));
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/processing"));
ENV["COLUMNS"] = 200;

In [2]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra

In [3]:
using NurseData
using ForecastData
using GeographicData

In [15]:
using ReusableResourceAllocation

In [5]:
ne_states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 4, 1)
end_date   = Date(2020, 5, 1)
travel_threshold_hours = 4.0;

In [6]:
nurses = num_nurses_from_beds(ne_states);
demand = compute_ihme_forecast(start_date, end_date, ne_states);
adj = compute_adjacencies(ne_states, hrs=travel_threshold_hours);

In [7]:
model = reusable_resource_allocation(
    nurses,
    zeros(Float32, size(demand)...),
    demand,
    adj,
    send_new_only=false,
    send_wait_period=10,
    verbose=true
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 4805 rows, 4752 columns and 1313371 nonzeros
Model fingerprint: 0x67dad969
Model has 330 SOS constraints
Variable types: 4752 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 1e+04]
Found heuristic solution: objective 6.820000e+11
Presolve added 502 rows and 0 columns
Presolve removed 0 rows and 1035 columns
Presolve time: 2.01s
Presolved: 5307 rows, 3717 columns, 324674 nonzeros
Presolved model has 330 SOS constraint(s)
Found heuristic solution: objective 4.811129e+11
Variable types: 3717 continuous, 0 integer (0 binary)

Root relaxation: objective 5.346088e+02, 592 iterations, 0.06 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

In [8]:
shortage_per_day = (i,t) -> max.(0,
    demand[i,t] - (
        nurses[i]
        - sum(sent[i,:,1:t])
        + sum(sent[:,i,1:t-1])
    )
)
shortage = i -> sum(shortage_per_day(i,t) for t=1:size(sent,3))
total_shortage = sum(shortage(i) for i=1:size(sent,1));

In [9]:
summary = DataFrame(
    state=ne_states,
    total_sent=sum(sent, dims=[2,3])[:],
    total_received=sum(sent, dims=[1,3])[:],
    shortage=shortage.(1:length(ne_states)),
)

,state,total_sent,total_received,shortage
,String,Float64,Float64,Float64
1,CT,3.3423e-7,3478.43,491.302
2,DE,541.334,0.0,39.3735
3,MA,2032.4,2793.38,218.289
4,MD,1464.41,0.0,72.2671
5,ME,573.981,17.7792,0.127244
6,NH,542.75,44.1178,18.2216
7,NJ,0.0,3806.59,61.3755
8,NY,6832.94,6060.34,1091.86
9,PA,4442.62,501.868,112.124


In [10]:
sent_matrix = DataFrame(sum(sent, dims=3)[:,:,1])
rename!(sent_matrix, Symbol.(ne_states))
insertcols!(sent_matrix, 1, :state => ne_states)

,state,CT,DE,MA,MD,ME,NH,NJ,NY,PA,RI,VT
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CT,0.0,0.0,0.0,0.0,0.0,0.0,1.67115e-7,1.67115e-7,0.0,0.0,0.0
2,DE,65.0546,0.0,0.0,0.0,0.0,0.0,460.37,0.0,0.0,15.9103,0.0
3,MA,0.0,0.0,0.0,0.0,0.0,0.0,726.32,1306.08,0.0,0.0,0.0
4,MD,0.0,0.0,0.0,0.0,0.0,0.0,604.878,859.535,0.0,0.0,0.0
5,ME,488.021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.9599,0.0
6,NH,2.15231,0.0,0.0,0.0,7.71198,0.0,0.0,532.886,0.0,0.0,0.0
7,NJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,NY,2893.89,0.0,2793.38,0.0,0.0,44.1178,0.0,0.0,501.868,599.68,0.0
9,PA,0.0,0.0,0.0,0.0,0.0,0.0,1376.22,3066.4,0.0,0.0,0.0


In [11]:
sent_vis_matrix = sum(sent, dims=3)[:,:,1] + diagm(nurses - sum(sent, dims=[2,3])[:])
sent_vis_matrix = DataFrame(sent_vis_matrix)
rename!(sent_vis_matrix, Symbol.(ne_states));
# CSV.write("nurse_sent_matrix.csv", sent_vis_matrix);

In [12]:
n_days = size(sent,3)
outcomes = DataFrame()
for (i,s) in enumerate(ne_states)
    single_state_outcome = DataFrame(
        state=fill(s, n_days),
        day=start_date .+ Dates.Day.(0:n_days-1),
        sent=sum(sent[i,:,:], dims=1)[:],
        received=sum(sent[:,i,:], dims=1)[:],
        total_nurses=[nurses[i] - sum(sent[i,:,1:t]) + sum(sent[:,i,1:t]) for t in 1:n_days],
        demand=demand[i,:],
        shortage=[shortage_per_day(i,t) for t in 1:n_days],
        sent_to=[sum(sent[i,:,t])>0 ? collect(zip(ne_states[sent[i,:,t] .> 0], sent[i,sent[i,:,t].>0,t])) : "[]" for t in 1:n_days],
        sent_from=[sum(sent[:,i,t])>0 ? collect(zip(ne_states[sent[:,i,t] .> 0], sent[sent[:,i,t].>0,i,t])) : "[]" for t in 1:n_days],
    )
    outcomes = vcat(outcomes, single_state_outcome)
end
# CSV.write("nurse_allocation_results.csv", outcomes)
println("First day:")
filter(row -> row.day == start_date, outcomes)

First day:


,state,day,sent,received,total_nurses,demand,shortage,sent_to,sent_from
,String,Date,Float64,Float64,Float64,Float32,Float64,Any,Any
1,CT,2020-04-01,3.3423e-7,0.0,2251.67,785.724,0.0,"[(""NJ"", 1.67115e-7), (""NY"", 1.67115e-7)]",[]
2,DE,2020-04-01,460.37,0.0,148.63,47.4493,0.0,"[(""NJ"", 460.37)]",[]
3,MA,2020-04-01,1306.08,0.0,3122.58,597.547,0.0,"[(""NY"", 1306.08)]",[]
4,MD,2020-04-01,1464.41,0.0,1386.92,575.461,0.0,"[(""NJ"", 604.878), (""NY"", 859.535)]",[]
5,ME,2020-04-01,0.0,0.0,639.333,35.5339,0.0,[],[]
6,NH,2020-04-01,532.886,0.0,108.781,30.0956,0.0,"[(""NY"", 532.886)]",[]
7,NJ,2020-04-01,0.0,1704.05,5703.39,3880.68,0.0,[],"[(""CT"", 1.67115e-7), (""DE"", 460.37), (""MD"", 604.878), (""RI"", 638.805)]"
8,NY,2020-04-01,0.0,6060.34,16051.7,10525.8,534.425,[],"[(""CT"", 1.67115e-7), (""MA"", 1306.08), (""MD"", 859.535), (""NH"", 532.886), (""PA"", 3066.4), (""VT"", 295.434)]"
9,PA,2020-04-01,3066.4,0.0,5160.93,892.099,0.0,"[(""NY"", 3066.4)]",[]


In [13]:
s = "NJ"
filter(row -> row.state == s, outcomes)

,state,day,sent,received,total_nurses,demand,shortage,sent_to,sent_from
,String,Date,Float64,Float64,Float64,Float32,Float64,Any,Any
1,NJ,2020-04-01,0.0,1704.05,5703.39,3880.68,0.0,[],"[(""CT"", 1.67115e-7), (""DE"", 460.37), (""MD"", 604.878), (""RI"", 638.805)]"
2,NJ,2020-04-02,0.0,0.0,5703.39,4655.39,0.0,[],[]
3,NJ,2020-04-03,0.0,0.0,5703.39,5475.14,0.0,[],[]
4,NJ,2020-04-04,0.0,484.768,6188.15,5703.39,0.0,[],"[(""PA"", 484.768)]"
5,NJ,2020-04-05,0.0,425.269,6613.42,6188.15,0.0,[],"[(""PA"", 425.269)]"
6,NJ,2020-04-06,0.0,466.18,7079.6,6613.42,0.0,[],"[(""PA"", 466.18)]"
7,NJ,2020-04-07,0.0,726.32,7805.92,7079.6,0.0,[],"[(""MA"", 726.32)]"
8,NJ,2020-04-08,0.0,0.0,7805.92,7814.22,8.2998,[],[]
9,NJ,2020-04-09,0.0,0.0,7805.92,7859.0,53.0757,[],[]


In [14]:
println("Connectivity:")
Dict(ne_states[i] => ne_states[row] for (i,row) in enumerate(eachrow(adj)))

Connectivity:


Dict{String,Array{String,1}} with 11 entries:
  "RI" => ["CT", "DE", "MA", "ME", "NH", "NJ", "NY", "VT"]
  "NJ" => ["CT", "DE", "MA", "MD", "NY", "PA", "RI"]
  "DE" => ["CT", "MD", "NJ", "NY", "PA", "RI"]
  "MD" => ["DE", "NJ", "NY", "PA"]
  "NH" => ["CT", "MA", "ME", "NY", "RI"]
  "NY" => ["CT", "DE", "MA", "MD", "NH", "NJ", "PA", "RI", "VT"]
  "ME" => ["CT", "MA", "NH", "RI", "VT"]
  "CT" => ["DE", "MA", "ME", "NH", "NJ", "NY", "RI", "VT"]
  "MA" => ["CT", "ME", "NH", "NJ", "NY", "RI", "VT"]
  "PA" => ["DE", "MD", "NJ", "NY"]
  "VT" => ["CT", "MA", "ME", "NY", "RI"]